In [1]:
# import custom modules wrote by julio
import importlib
import numpy as np
import seaborn as sns
%matplotlib inline
from capstone_01 import clean_data
from capstone_01 import inferential_statistics
import pandas as pd
# reload modules without restartign the kernall (it drives me crazy)
importlib.reload(clean_data);
importlib.reload(inferential_statistics);

In [2]:
# load data and cleant it
file = './raw_data/clinical/I-SPY_1_All_Patient_Clinical_and_Outcome_Data.xlsx'
df = clean_data.clean_my_data(file)
df.head(2)

,age,White,ER+,PR+,HR+,Bilateral,Right_Breast,MRI_LD_Baseline,MRI_LD_1_3dAC,MRI_LD_Int_Reg,MRI_LD_PreSurg,Alive,Survival_length,RFS,RFS_code,PCR,RCB
SUBJECTID,,,,,,,,,,,,,,,,,
1001,38.73,Yes,Yes,No,Yes,No,No,88,78,30,14,No,1264,751,1,No,2
1002,37.79,Yes,Yes,Yes,Yes,No,Yes,29,26,66,16,No,1155,1043,1,No,3


In [3]:
MRI_meta = '/Users/Julio/GitHub/SpringBoard/Projects/19-Captsone_01/raw_data/MRI/ISPY-1-Meta_all.csv'
MRI_df = pd.read_csv( './raw_data/MRI/ISPY-1-Meta_all.csv')
MRI_df = MRI_df.drop('Collection', axis = 1)
col = ['ID','Date','Descr','Mode','Series','Manufac','Model','SoftVer','SeriesID']
MRI_df.columns = col

DF = MRI_df.groupby(['ID','Manufac','Date']).count().reset_index()[['ID','Manufac']].drop_duplicates()
DF = DF.reset_index()[['ID','Manufac']]
DF['SUBJECTID']= DF['ID'].apply( lambda code: code[-4:] )
DF['SUBJECTID']= DF['ID'].apply( lambda code: code[-4:] )
DF['SUBJECTID'] = DF['SUBJECTID'].astype('int64')

DF = DF.drop('ID', axis = 1)
DF['Manufac'] = DF['Manufac'].apply( lambda x: x.lower() )


def to_words(string):
    if string == 'ge medical systems':
        string = 'Yes'
    else:
        string = 'No'
    return string

DF['Manufac'] = DF.Manufac.apply(to_words)
DF.columns = ['GE','SUBJECTID']
DF.head(2)

,GE,SUBJECTID
0,Yes,1001
1,Yes,1002


In [4]:
type(df.reset_index()['SUBJECTID'][0])

numpy.int64

In [5]:
df2 = DF.merge(df.reset_index()).set_index('SUBJECTID')
cols = ['GE','White', 'ER+','PR+', 'HR+', 'Bilateral','Right_Breast','PCR','Alive']
df = df2[cols]
df.head()

,GE,White,ER+,PR+,HR+,Bilateral,Right_Breast,PCR,Alive
SUBJECTID,,,,,,,,,
1001,Yes,Yes,Yes,No,Yes,No,No,No,No
1002,Yes,Yes,Yes,Yes,Yes,No,Yes,No,No
1003,Yes,Yes,Yes,Yes,Yes,No,No,No,Yes
1008,Yes,Yes,Yes,Yes,Yes,Yes,Yes,No,Yes
1009,Yes,No,No,No,No,No,No,No,Yes


## Inferential_statistics
### 1. Categorical vs Categorical

In [6]:
# assign all categorical predictors
predictors = df.columns[0:2]
predictors

Index(['GE', 'White'], dtype='object')

In [7]:
# example of contingency table
inferential_statistics.contingency_table('GE', 'White',df)


White,Yes,No
GE,,
Yes,89,25
No,52,18


In [8]:
inferential_statistics.categorical_data('Alive', ['White','GE'], df)

,p-value,Relative_Risk,RR_lb,RR_ub
White,0.485388,1.0812,0.9025,1.2954
GE,0.864929,0.9740,0.8473,1.1196


In [9]:
# example of contingency table
inferential_statistics.contingency_table('White', 'Alive',df)

Alive,Yes,No
White,,
Yes,117,24
No,33,10


In [10]:
# run chi-squared test on Survival as outcome
inferential_statistics.categorical_data('Alive', predictors, df)

,p-value,Relative_Risk,RR_lb,RR_ub
GE,0.864929,0.9740,0.8473,1.1196
White,0.485388,1.0812,0.9025,1.2954


## 2. Continous vs Ca## 2. Continous vs Categorical (ANOVA)
### 2.1 Effect of Age on PCA and Survivaltegorical (ANOVA)
### 2.1 Effect of Age on PCA and Survival

In [11]:
predictor= ['age']
outcome = 'PCR'
anova_table, OLS = inferential_statistics.linear_models(df, outcome, predictor);
sns.boxplot(x= outcome, y=predictor[0], data=df, palette="Set3");

KeyError: "['age'] not in index"

In [ ]:
predictor= ['age']
outcome = 'Alive'
anova_table, OLS = inferential_statistics.linear_models(df, outcome, predictor);
sns.boxplot(x= outcome, y=predictor[0], data=df, palette="Set3");

### explore interactions between age, survival, and PCR

In [ ]:
sns.boxplot(x= 'PCR', y='age', hue ='Alive',data=df, palette="Set3");

## The survival of patients that achieved PCR is affected by their age

In [ ]:
predictor= ['age']
outcome = 'Alive'
anova_table, OLS = inferential_statistics.linear_models(df.loc[df.PCR=='Yes',:], outcome, predictor);
sns.boxplot(x= outcome, y=predictor[0], data=df.loc[df.PCR=='Yes',:], palette="Set3");

In [ ]:
sns.boxplot(x= 'PCR', y='Survival_length', hue ='Alive',data=df, palette="Set3");

### 3.1 Effect of MRI measurements on PCR

In [ ]:
R = inferential_statistics.anova_MRI('PCR', df);

### 3.1 Effect of MRI measurements on Survival

In [ ]:
R = inferential_statistics.anova_MRI('Alive', df);